In [28]:
import pandas as pd 
import numpy as np

In [63]:
df = pd.read_csv('NYPD_Motor_Vehicle_Collisions.csv', index_col = 0, low_memory = False)

In [64]:
df.shape

(1149041, 28)

In [65]:
df.columns

Index(['TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'LOCATION',
       'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
       'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
       'CONTRIBUTING FACTOR VEHICLE 5', 'UNIQUE KEY', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4',
       'VEHICLE TYPE CODE 5'],
      dtype='object')

In [66]:
df['VEHICLE TYPE CODE 1'] = df['VEHICLE TYPE CODE 1'].fillna('')
df['VEHICLE TYPE CODE 2'] = df['VEHICLE TYPE CODE 2'].fillna('')
df['VEHICLE TYPE CODE 3'] = df['VEHICLE TYPE CODE 3'].fillna('')
df['VEHICLE TYPE CODE 4'] = df['VEHICLE TYPE CODE 4'].fillna('')
df['VEHICLE TYPE CODE 5'] = df['VEHICLE TYPE CODE 5'].fillna('')

In [67]:
np.unique(df['VEHICLE TYPE CODE 1'])

array(['', '3D', 'AM', 'AMBUL', 'AMBULANCE', 'AR', 'Ambul', 'BA', 'BACKH',
       'BICYCLE', 'BOBCA', 'BOX T', 'BR', 'BU', 'BUS', 'CB', 'CM', 'CO',
       'COMME', 'CONV', 'Car C', 'Comme', 'Crane', 'DELIV', 'DP', 'DS',
       'DUMP', 'EBIKE', 'ELECT', 'FB', 'FDNY', 'FIRE TRUCK', 'FIRET',
       'FLAT', 'FORKL', 'FR', 'Ford', 'G OMR', 'GE/SC', 'GG', 'Ice C',
       'LARGE COM VEH(6 OR MORE TIRES)', 'LF', 'LIVERY VEHICLE', 'LL',
       'LW', 'MACK', 'MB', 'MD', 'MOTOR', 'MOTORCYCLE', 'MS', 'OP',
       'OTHER', 'PASSENGER VEHICLE', 'PEDICAB', 'PICK-UP TRUCK', 'PK',
       'PM', 'POSTA', 'PSR', 'Pavin', 'RF', 'RV', 'Road', 'SCHOO',
       'SCOOTER', 'SEMI', 'SMALL COM VEH(4 TIRES) ',
       'SPORT UTILITY / STATION WAGON', 'SPRIN', 'ST', 'ST150', 'STREE',
       'Scoot', 'TAN P', 'TAXI', 'TK', 'TN', 'TOW T', 'TR', 'TRACT',
       'TRAIL', 'TRUCK', 'TT', 'UNKNOWN', 'UTIL', 'UTILI', 'Util', 'VAN',
       'VAN T', 'VC', 'VN', 'VT', 'Van', 'ambul', 'bus', 'east', 'horse',
       'sanit', 'sc

In [69]:
df['all_vehicle'] = list(map(lambda a,b,c,d,e: a + b + c + d + e, 
                             df['VEHICLE TYPE CODE 1'],
                            df['VEHICLE TYPE CODE 2'],
                            df['VEHICLE TYPE CODE 3'],
                            df['VEHICLE TYPE CODE 4'],
                            df['VEHICLE TYPE CODE 5']))

In [70]:
df['bike_involved'] = list(map(lambda x: 'BICYCLE' in x, df['all_vehicle']))

In [72]:
df['CONTRIBUTING FACTOR VEHICLE 1'] = df['CONTRIBUTING FACTOR VEHICLE 1'].fillna('')
df['CONTRIBUTING FACTOR VEHICLE 2'] = df['CONTRIBUTING FACTOR VEHICLE 2'].fillna('')
df['CONTRIBUTING FACTOR VEHICLE 3'] = df['CONTRIBUTING FACTOR VEHICLE 3'].fillna('')
df['CONTRIBUTING FACTOR VEHICLE 4'] = df['CONTRIBUTING FACTOR VEHICLE 4'].fillna('')
df['CONTRIBUTING FACTOR VEHICLE 5'] = df['CONTRIBUTING FACTOR VEHICLE 5'].fillna('')

In [78]:
df['all_factors'] = list(map(lambda a,b,c,d,e: a +" " + b + " " +c + " " + d + " " + e, 
                             df['CONTRIBUTING FACTOR VEHICLE 1'],
                            df['CONTRIBUTING FACTOR VEHICLE 2'],
                            df['CONTRIBUTING FACTOR VEHICLE 3'],
                            df['CONTRIBUTING FACTOR VEHICLE 4'],
                            df['CONTRIBUTING FACTOR VEHICLE 5']))

In [82]:
sum(list(map(lambda x:  'Bicyclist' in x, df.all_factors)))

2342

In [83]:
factors = {'physical_condition': ['Fatigued/Drowsy', 'Fell Asleep', 
                                  'Illness',  'Lost Consciousness', 
                                  'Physical Disability',  'Prescription Medication',  
                                  'Alcohol Involvement','Drugs (Illegal)'], 
           
           'car_condition': ['Accelerator Defective',
                             'Brakes Defective', 'Headlights Defective',
                             'Steering Failure', 'Tow Hitch Defective', 
                             'Windshield Inadequate',
                             'Tire Failure/Inadequate', 'Other Vehicular'],
           
           'Road_condition': ['Lane Marking Improper/Inadequate','Obstruction/Debris', 
                              'Other Lighting Defects', 'Pavement Defective', 
                              'Pavement Slippery', 'Shoulders Defective/Improper',
                              'Traffic Control Device Improper/Non-Working'], 
           
           'Poor_driving': ['Aggressive Driving/Road Rage', 'Backing Unsafely',
                                   'Cell Phone (hand-held)', 'Cell Phone (hands-free)', 
                                   'Driver Inattention/Distraction',  
                                   'Driver Inexperience', 'Failure to Keep Right',
                                   'Failure to Yield Right-of-Way','Following Too Closely', 
                                   'Other Electronic Device', 'Outside Car Distraction',
                                   'Oversized Vehicle', 'Passenger Distraction', 
                                   'Passing or Lane Usage Improper', 
                                   'Reaction to Other Uninvolved Vehicle', 
                                   'Traffic Control Disregarded', 'Turning Improperly', 
                                   'Unsafe Lane Changing', 'Unsafe Speed'], 
           
           'Other': ['Unspecified','View Obstructed/Limited',  
                     'Animals Action', 'Driverless/Runaway Vehicle', 
                     'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion'] 
          }
